In [1]:
!pip install python-terrier

     |████████████████████████████████| 95 kB 2.3 MB/s 
     |████████████████████████████████| 1.1 MB 16.4 MB/s 
     |████████████████████████████████| 69 kB 6.5 MB/s 
     |████████████████████████████████| 255 kB 46.0 MB/s 
     |████████████████████████████████| 294 kB 47.5 MB/s 
     |████████████████████████████████| 126 kB 40.8 MB/s 
     |████████████████████████████████| 596 kB 33.6 MB/s 
     |████████████████████████████████| 72 kB 1.3 MB/s 
     |████████████████████████████████| 1.8 MB 24.4 MB/s 
     |████████████████████████████████| 6.3 MB 39.3 MB/s 
     |████████████████████████████████| 291 kB 46.6 MB/s 
     |████████████████████████████████| 45 kB 2.2 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.7.1-py3-none-any.whl size=102452 sha256=6987c2f3a179a14b445ec743988c6eb56c72a6d6ef980eed95aac94278c354f7
  Stored in directory: /root/.cache/pip/wheels/c0/9a/c8/1c2d9ec6a1494bb54f47e0d2627b5ed7b2de704b66723d3417
  Created wheel for ir-measures: filen

In [2]:
import pyterrier as pt
import pandas as pd
import os
from pyterrier.measures import *
import re

In [3]:
foodresults = pd.read_csv("foodresults.csv", encoding="latin-1")
index_result = pd.read_csv("index_result.csv")
df = pd.read_csv("dataset_with_relevance.csv",encoding="latin-1")

In [4]:
if not pt.started():
  pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.7.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


In [5]:
import re
s="sdfgfs$%$#%#$9998 sdfg"
word1 = " ".join(re.findall("[a-zA-Z]+", s))
word1

'sdfgfs sdfg'

In [6]:
data=[]
for index,row in df.iterrows():
  query = " ".join(re.findall("[a-zA-Z]+", row["query"]))
  recipe_id = row["recipe_id"]
  recipe_name = " ".join(re.findall("[a-zA-Z]+", row["recipe_name"]))
  Relevance = row["Relevance"]
  ingredients = " ".join(re.findall("[a-zA-Z]+", str(row["ingredients"])))
  recipe = " ".join(re.findall("[a-zA-Z]+", row["recipe"]))
  rate = row["rate"]
  views = row["views"]
  description = " ".join(re.findall("[a-zA-Z]+", row["description"]))
  keywords = " ".join(re.findall("[a-zA-Z]+", str(row["keywords"])))
  calories = row["calories"]
  fatContent = row["fatContent"]
  proteinContent = row["proteinContent"]
  data.append([query,recipe_id,recipe_name,Relevance,ingredients,recipe,
               rate,views,description,keywords,calories,fatContent,proteinContent])
new_df = pd.DataFrame(data,columns=["query","docno","recipe_name","Relevance","ingredients","recipe",
               "rate","views","description","keywords","calories","fatContent","proteinContent"])

In [7]:
new_df.head(6)

,query,docno,recipe_name,Relevance,ingredients,recipe,rate,views,description,keywords,calories,fatContent,proteinContent
0,tomato,6492,Macaroni and Tomatoes,5,large potatoes teaspoon salt eggs lightly beat...,Scrub potatoes well but do not peel In a sauce...,4.5,6,Make and share this Potato Plum Dumplings reci...,Vegetable Hungarian European Low Protein Healt...,270.6,8.9,5.8
1,tomato,50060,Tofu Stuffed Cherry Tomatoes,5,onion diced garlic cloves minced carrot diced ...,Sweat onion and garlic in small amount of oil ...,0.0,0,Creamy fat free low cal and a serving of veggi...,Lactose Free Low Protein Low Cholesterol Egg F...,107.6,0.8,3.8
2,tomato,54375,Canned Tomatoes Vine Ripened Tomatoes in Tomat...,5,butter recipe cake mix baked in a greased x cm...,For best results bake the cake the day before ...,0.0,0,Make and share this Lamingtons recipe from Foo...,Low Protein Low Cholesterol Healthy Kid Friend...,179.2,3.1,1.3
3,tomato,11958,Chilli Tomato Sauce Tomato Salsa,5,large mushrooms cup breadcrumbs cup parmesan c...,clean mushrooms with damp cloth remove stems a...,5.0,1,Make and share this Moms Stuffed Mushrooms rec...,Healthy Mins For Large Groups,16.8,0.7,1.4
4,tomato,51985,Cherry Tomato Planets,5,teaspoons sesame oil tablespoons gingerroot sh...,Place the sesame oil in a hot wok with the gin...,0.0,0,This is from the Donna Hay cookbook quot Cooki...,Asian Low Protein Low Cholesterol Healthy Mins...,86.8,2.3,0.5
5,tomato,16912,BLANCHING TOMATOES for COOKING amp CANNING,4,ounces large shrimp peeled and deveined tables...,Heat the oil in a large heavy bottomed skillet...,0.0,0,Make and share this Cilantro Lime Shrimp recip...,Very Low Carbs Healthy Mins,165.6,5.4,24.6


In [8]:
index_dir = "./docs_index"
indexer = pt.DFIndexer(index_dir, overwrite=True,blocks=True)
index_ref = indexer.index(new_df["recipe"], new_df["docno"])
index_ref.toString()

JavaException: ignored

In [ ]:
index = pt.IndexFactory.of(index_ref)

In [9]:

data=[]
foodresults["docno"] = range(0,len(foodresults))
foodresults["recipe_id"] = range(0,len(foodresults))
foodresults.head(5)
for index,row in foodresults.iterrows():
  docno = "d"+str(row["docno"])
  recipe_id = str(row["recipe_id"])
  recipe_name = " ".join(re.findall("[a-zA-Z]+", row["name"]))
  ingredients = " ".join(re.findall("[a-zA-Z]+", str(row["ingredients"])))
  recipe = " ".join(re.findall("[a-zA-Z]+", str(row["recipe"])))
  rate = row["rate"]
  views = row["views"]
  description = " ".join(re.findall("[a-zA-Z]+", row["description"]))
  keywords = " ".join(re.findall("[a-zA-Z]+", str(row["keywords"])))
  calories = row["calories"]
  fatContent = row["fatContent"]
  proteinContent = row["proteinContent"]
  data.append([recipe_id,recipe_name,ingredients,recipe,
               rate,views,description,keywords,calories,fatContent,proteinContent])
whole_df = pd.DataFrame(data,columns=["docno","recipe_name","ingredients","recipe",
               "rate","views","description","keywords","calories","fatContent","proteinContent"])

In [10]:
whole_df.head(5)

,docno,recipe_name,ingredients,recipe,rate,views,description,keywords,calories,fatContent,proteinContent
0,0,Chinese Fried Rice,cup finely chopped onion tablespoons oil egg l...,Heat tbsp oil in wok add chopped onions and st...,5.0,541,This Chinese fried rice has the flavor those o...,Pork Poultry Rice Meat Chinese Asian Low Chole...,497.8,16.3,22.3
1,1,Perfect Pork Tenderloin,lb boneless pork tenderloin salt and pepper dr...,NOTE nbsp The success of this cooking method w...,5.0,434,My way to cook a pork tenderloin or a boneless...,Meat Very Low Carbs High Protein Healthy High ...,136.2,4.0,23.4
2,2,Pulled Pork Crock Pot,lbs pork roast shoulder or butt large onions c...,Slice one onion and place in crock pot Put in ...,5.0,642,I found this pulled pork recipe years ago and ...,Meat Healthy Beginner Cook Easy,335.1,6.9,40.2
3,3,The Most Wonderful Gingerbread Cookies,cups all purpose flour teaspoons baking powder...,In a small bowl whisk together flour baking po...,4.5,559,This is my very favorite gingerbread cookie re...,Cookie Brownie Healthy Christmas Oven Hours,134.2,3.2,1.9
4,4,Restaurant Teriyaki Sauce,cup soy sauce cup water teaspoon ground ginger...,Mix all but cornstarch and c water in a sauce ...,5.0,341,You know the nice sweet thick Teriyaki sauce y...,Low Protein Low Cholesterol Healthy Mins Easy,290.5,0.1,5.3


In [60]:
whole_index_dir = "./whole_docs_index"
whole_indexer = pt.DFIndexer(whole_index_dir, overwrite=True)
whole_index_ref = whole_indexer.index(whole_df["recipe"], whole_df["docno"],
                    whole_df["recipe_name"], whole_df["keywords"], 
                    whole_df["description"], whole_df[""])
whole_index_ref.toString()

'./whole_docs_index/data.properties'

In [61]:
whole_index = pt.IndexFactory.of(whole_index_ref)

In [13]:
topics = pd.read_csv("topics.csv")
qrels = pd.read_csv("qrel.csv")
qrels = qrels.drop(columns=["iteration"])

In [14]:
topics.head(5)

,qid,query
0,1,apple
1,2,bacon
2,3,baking soda
3,4,barbecue ribs
4,5,beef


In [15]:
topics["qid"] = topics["qid"].astype(str)

In [16]:
topics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     30 non-null     object
 1   query   30 non-null     object
dtypes: object(2)
memory usage: 608.0+ bytes


In [17]:
qrels["qid"] = qrels["qid"].astype(str)
qrels["docno"] = qrels["docno"].astype(str)

**Machine Learning**

In [18]:
# constants
RANK_CUTOFF = 100


In [62]:
bm25 = pt.BatchRetrieve(whole_index, wmodel="BM25")
tfidf = pt.BatchRetrieve(whole_index, wmodel="TF_IDF")
sdm = pt.rewrite.SDM()
ltr_feats_new1 = (bm25 % RANK_CUTOFF) >> pt.text.get_text(whole_index, ["recipe_name","keywords","description"]) >> (
    pt.transformer.IdentityTransformer()
    ** # score of title (not originally indexed)
    (pt.text.scorer(body_attr="recipe_name", takes='docs', wmodel='BM25'))
    ** # score of title (not originally indexed)
    (pt.text.scorer(body_attr="keywords", takes='docs', wmodel='BM25'))
    ** 
    (pt.text.scorer(body_attr="description", takes='docs', wmodel='BM25'))
)
# for reference, lets record the feature names here too
fnames=["f1","f2"]

In [63]:
from sklearn.ensemble import RandomForestRegressor
pipeline = pt.FeaturesBatchRetrieve(whole_index, wmodel="BM25", features=["WMODEL:Tf"])
rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=42, n_jobs=2)
rf_pipe = ltr_feats_new1 >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(topics, qrels)


16:58:03.233 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
16:58:05.216 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
16:58:07.382 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    1.0s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    1.9s finished


In [65]:
pt.Experiment(
    [rf_pipe, bm25 % 50, tfidf % 50],
    topics,
    qrels,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"],
    names=["our model","bm25","tfidf"],
    baseline = 2
)

16:59:19.486 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
16:59:21.545 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
16:59:23.632 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    0.2s finished


,name,map,ndcg,ndcg_cut_10,mrt,map +,map -,map p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,our model,0.600094,0.695757,0.929300,339.534932,28.0,0.0,5.133588e-14,29.0,0.0,2.994684e-15,28.0,0.0,0.000002
1,bm25,0.359174,0.490354,0.796588,96.799355,3.0,3.0,2.980631e-01,4.0,3.0,8.833624e-01,2.0,2.0,0.493680
2,tfidf,0.359793,0.490466,0.795657,91.096616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df = pd.DataFrame(
    [
        ["q1", "chemical reactions", "d1", "professor protor poured the chemicals"],
        ["q1", "chemical reactions", "d2", "chemical brothers turned up the beats"],
    ], columns=["qid", "query", "docno", "text"])
textscorerTf = pt.text.scorer(body_attr="recipe_name", wmodel="Tf")
rtr = textscorerTf.transform(whole_df)

KeyError: ignored

In [25]:
rtr

,qid,docno,text,rank,score,query
0,q1,d1,professor protor poured the chemicals,0,1.0,chemical reactions
1,q1,d2,chemical brothers turned up the beats,1,1.0,chemical reactions
